# Bioluminescence image processing in python 2.7

abel 25 feb 2018 \
latest update: 11 Mar 2018

The purpose of this file is to draw on python tools for analysis of circadian bioluminescence reporters. To run this file:
   1. edit the contents of cell 2, then do 
   2. `Kernel > Restart and run all`. This will yield raw, detrended, and detrended+denoised data in the same folder as the inputs. It will also identify relevant circadian features.
Each step is annotated to explain its process. 

Dependencies: numpy, scipy, matplotlib, pandas>=0.22.0, spectrum

Code by JH Abel and Y Shan. Step by step instructions follow. Portions of the local imports were written by P.C. St. John and published in https://doi.org/10.1016/j.bpj.2014.10.026. Currently, this file is unpublished, but we may include it in the supplement to a publication.

contact: jhabel01 (at) gmail (dot) com for further information on how to use this script.

### Step-by-step processing
-----------------------------

1. **Import the data.**
    The data may be imported directly from the imageJ output .xls file. If so, set:
    ```
    pull_from_imagej = True
    input_folder = [path to folder containing data]
    input_ij_file   = [name of imageJ file with no extension]
    data_type = [type of data being imported]
    ```
    Note that data_type will be used to name all future outputs of the file. For example, the raw signal and raw XY tracking will be named:
    ```
    raw_signal = output_folder +data_type+'_signal'+output_extension
    raw_xy = output_folder +data_type+'_XY'+output_extension
    ```
    If the data is _not_ being taken from an imageJ file but has already been converted into `_signal.csv` and
    `_XY.csv` files, one need only adjust `data_type` so that `raw_signal` and `raw_xy` look in the correct
    locations. The flag for pulling from imageJ should be set to False.<br><br>
    
2. **Interpolate missing intermediate values in the dataset.**
    Outliers > $5\sigma$ are first removed. Note that interpolation is linear and only applied to intermediate points (i.e., no extrapolation is performed).<br><br>

3. **Detrending via Hodrick-Prescott filter.**
    An HP filter is applied to detrend the data. We (JH Abel and Y Shan) found that this performs better than polynomial detrending in that it does not overfit the trend and begin to fit the oscillatory components. Parameter selection for the filter is explained in the code.<br><br>
    
4. **Eigendecomposition and reconstruction of the signal to denoise the data.**
    An eigendecomposition of the autocorrelation matrix is performed. Any eigenvectors with eigenvalue $>5$% of the total sum of all eigenvalues is kept, and the signal is reconstructed from these eigenvectors. The process is theoretically explained here: http://www.fil.ion.ucl.ac.uk/~wpenny/course/subspace.pdf<br>
     **Then truncate first 12 h to remove artifact.**
    All trajectories truncated to start at 12 h. If a cell is first found after 12 h, no time is truncated since it does not exhibit an artifact from plating.<br><br>
    
5. **Lomb-scargle periodogram to determine which cells have oscillation.**
    The Lomb-Scargle periodogram is used to identify statistically significant circadian oscillation. Here,
    we have defined this as having a dominant peak of $P<0.05$ between periods of 18 and 30 h. The method for 
    periodogram calculation and corresponding P-values is from WH Press, Numerical Recipes, 1994, p576. The final periodogram is normalized using the standard normalization in Astropy or Scipy. We applied this to the detrended, denoised, and truncated data so that the noise difference between channels is corrected, and no statistical difference in peak distribution exists within a WT AVPCre SCN between channels (t-test, $P=0.75$). See `lomb_testing.py` for details.<br><br>
    
6. **Fit a damped sinusoid to the data to yield sine fit, phase, amplitude, damping rate, goodness of fit.**
    All rhythmic cells are now fit by a damped sinusoid plus polynomial. Only the sinusoid data is returned.<br><br>
    
7. **Export data from the analyses.** 
    The data produced at most steps of this process is then saved in the output folder. 
    
|    Suffix | Data        |
|-----------|-------------|
|`_signal.csv`| Raw bioluminescence.|
|`_XY.csv`| Raw location values.|
| `_signal_detrend.csv`| Data interpolated and detrended via HP filter.|
| `_signal_detrend_denoise.csv`| Detrended, denoised, 12h truncated data. | 
|`_XY_detrend_denoise.csv` | Detrended, denoised, 12h truncated locations. | 
|`_zscore.csv` | z-scored detrended and denoised data |
|`_lombscargle.csv`| Normalized Lomb-Scargle periodogram values. |
|`_cosine.csv`| The sinusoid fit to each cell. Only created if cell is rhythmic.|
|`_cosine_phases.csv`| The sinusoid phases corresponding to the fit.|
|`_oscillatory_params.csv`| Rhythmic, estimated phase, normalized LS circadian peak, period (h), amplitude, decay, $R^2$.|
<br><br>

**Lastly, the code generates summary plots at random.**
    Summaries of the data processing are performed for three cells selected at random. Subpanels are A-F, left to right, top to bottom. (A) Raw bioluminescence data. (B) Raw bioluminescence with trend (gold). (C) Detrended bioluminescence. (D) Eigendecomposition (gold) with threshold for reconstruction (red). (E) Detrended and denoised reconstructed signal. (F) Lomb-Scargle periodogram with rhythmic test in y-axis. (G) Sinusoid fit to data with $R^2$ value in y-axis label. (H) Detrended, denoised, and sine fit plotted simultaneously.

In [ ]:
#Python package IMPORTS
#Load the excel data and output the relevant portions to a CSV file.
#This follows Yongli's conventions for how excel datasheets are laid out.


from __future__ import division
# imports
import numpy  as np
import scipy as sp
import pandas as pd
from matplotlib import gridspec
import matplotlib.pyplot as plt

from LocalImports import PlotOptions as plo
from LocalImports import Bioluminescence as blu
from LocalImports import DecayingSinusoid as dsin
from LocalImports import ProcessingFunctions_20180326 as pf

### Editable content in the following cell:

In [ ]:
#inputs nms pre
pull_from_imagej = False
input_folder = 'Data/scn2_VIPBmal1KO_20170909_SGLE1/' # edit this
input_ij_extension = '.csv'# edit this

# do we want plots?
plotcount = 2
# for each dataset, this is formatted [descriptor, root filename, color ('Red', or 'Green')]
input_ij_file_type   = [['Pre_Green', '090917VIPMOP_Pre_Green', 'Green'],
                        ['Pre_Green', '090917VIPMOP_Pre_Red', 'Red'],
                        ['TTX_Red', '090917VIPMOP_TTX_Red', 'Red'],
                        ['TTX_Green', '090917VIPMOP_TTX_Green', 'Green'],
                        ['Wash Red', '090917VIPMOP_Wash_Red', 'Red'],
                        ['Wash_Green', '090917VIPMOP_Wash_Green', 'Green']
                        ] # edit this

# list all the datasets
all_inputs=[]
for input_ij in input_ij_file_type:
    all_inputs.append(pf.generate_filenames_dict(input_folder, input_ij[1], 
                                    pull_from_imagej, input_ij[0], 
                                    input_ij_extension, input_ij[2]))

### Generally, do not edit content below this line. It is already optimized for data processing.
------------

The following cell does the collection and saving of the raw data. If the raw data is already saved, the pull_from_imagej flag should be set to false, so that this cell will not run.

Note: pulling from ImageJ uses about 1.4Gb ram.

In [ ]:
# process the data for every set of inputs
for files_dict in all_inputs:
    
    # assign all filenames to correct local variables
    data_type = files_dict['data_type']
    input_data = files_dict['input_data']
    input_folder = files_dict['input_folder']
    input_ij_extension = files_dict['input_ij_extension']
    input_ij_file = files_dict['input_ij_file']
    output_cosine = files_dict['output_cosine']
    output_cosine_params = files_dict['output_cosine_params']
    output_detrend = files_dict['output_detrend']
    output_zscore = files_dict['output_zscore']
    output_detrend_smooth = files_dict['output_detrend_smooth']
    output_detrend_smooth_xy = files_dict['output_detrend_smooth_xy']
    output_pgram = files_dict['output_pgram']
    output_phases = files_dict['output_phases']
    pull_from_imagej = files_dict['pull_from_imagej']
    raw_signal = files_dict['raw_signal']
    raw_xy = files_dict['raw_xy']
    color=files_dict['color']
    

    # does the actual processing of the data
    # I. IMPORT DATA
    # only perform this step if pull_from_imagej is set to True
    if pull_from_imagej:
        pf.load_imagej_file(input_data, raw_signal, raw_xy, color=color)

    raw_times, raw_data, locations, header = pf.import_data(raw_signal, raw_xy)

    # II. INTERPOLATE MISSING PARTS
    # truncate 0 h and interpolate
    interp_times, interp_data, locations = pf.truncate_and_interpolate(
        raw_times, raw_data, locations, truncate_t=0)

    # III. DETREND USING HP Filter
    #(Export data for presentation of raw tracks with heatmap in Prism.)
    detrended_times, detrended_data, trendlines = pf.hp_detrend(
                                        interp_times, interp_data)

    # IV. SMOOTHING USING EIGENDECOMPOSITION
    # eigendecomposition
    denoised_times, denoised_data, eigenvalues = pf.eigensmooth(detrended_times,
        detrended_data, ev_threshold=0.05, dim=40)
    # TRUNCATE 12 INITIAL HOURS
    final_times, final_data, locations = pf.truncate_and_interpolate(denoised_times,
                                    denoised_data, locations, truncate_t=12)

    # V. LS PERIODOGRAM TEST FOR RHYTHMICITY
    lspers, pgram_data, circadian_peaks, lspeak_periods, rhythmic_or_not = pf.LS_pgram(final_times, final_data)

    # VI. GET A SINUSOIDAL FIT TO EACH CELL
    # use final_times, final_data
    # use forcing to ensure period within 1h of LS peak period
    sine_times, sine_data, phase_data, refphases, periods, amplitudes, decays, r2s, meaningful_phases =\
         pf.sinusoidal_fitting(final_times, final_data, rhythmic_or_not, 
                               fit_times=raw_times, forced_periods=lspeak_periods)
    # get metrics
    circadian_metrics = np.vstack([rhythmic_or_not, circadian_peaks, refphases, periods, amplitudes,
                                   decays, r2s])

    # VII. SAVING ALL COMPONENTS
    timer = plo.laptimer()
    print "Saving data... time: ",

    # detrended
    cell_ids = header[~np.isnan(header)]
    output_array_det = np.nan*np.ones((len(detrended_times)+1, len(cell_ids)+2))
    output_array_det[1:,0] = detrended_times
    output_array_det[1:,1] = np.arange(len(detrended_times))
    output_array_det[0,2:] = refphases
    output_array_det[1:,2:] = detrended_data
    output_df = pd.DataFrame(data=output_array_det,
            columns = ['TimesH', 'Frame']+list(cell_ids))
    output_df.loc[0,'Frame']='RefPhase'
    output_df.to_csv(output_detrend, index=False)
    del output_df # clear it

    # detrended-denoised
    output_array = np.nan*np.ones((len(final_times)+1, len(cell_ids)+2))
    output_array[1:,0] = final_times
    output_array[1:,1] = np.arange(len(final_times))
    output_array[0,2:] = refphases
    output_array[1:,2:] = final_data
    output_df = pd.DataFrame(data=output_array,
            columns = ['TimesH', 'Frame']+list(cell_ids))
    output_df.loc[0,'Frame']='RefPhase'
    output_df.to_csv(output_detrend_smooth, index=False)
    del output_df # clear it
    
    # Z-Score
    output_array = np.nan*np.ones((len(final_times)+1, len(cell_ids)+2))
    output_array[1:,0] = final_times
    output_array[1:,1] = np.arange(len(final_times))
    output_array[1:,2:] = sp.stats.zscore(final_data, axis=0, ddof=0)
    output_df = pd.DataFrame(data=output_array,
            columns = ['TimesH', 'Frame']+list(cell_ids))
    output_df.loc[0,'Frame']='RefPhase'
    output_df.loc[0,list(cell_ids)]=refphases
    output_df.to_csv(output_zscore, index=False)
    del output_df # clear it

    # LS Pgram
    output_array = np.nan*np.ones((len(lspers), len(pgram_data[0,:])+1))
    output_array[:,0] = lspers
    output_array[:,1:] = pgram_data
    output_df = pd.DataFrame(data=output_array,
            columns = ['LSPeriod']+list(cell_ids))
    output_df.to_csv(output_pgram, index=False)
    del output_df # clear it

    #sinusoids
    output_array = np.nan*np.ones((len(sine_times), len(cell_ids)+2))
    output_array[:,0] = sine_times
    output_array[:,1] = np.arange(len(sine_times))
    output_array[:,2:] = sine_data
    output_df = pd.DataFrame(data=output_array,
            columns = ['TimesH', 'Frame']+list(cell_ids))
    output_df.to_csv(output_cosine, index=False)
    del output_df

    #phases
    output_array = np.nan*np.ones((len(sine_times), len(cell_ids)+2))
    output_array[:,0] = sine_times
    output_array[:,1] = np.arange(len(sine_times))
    output_array[:,2:] = phase_data
    output_df = pd.DataFrame(data=output_array,
            columns = ['TimesH', 'Frame']+list(cell_ids))
    output_df.to_csv(output_phases, index=False)
    del output_df

    # sinusoid parameters and XY locations
    # this gets the locations for each cell by just giving their mean
    # location and ignoring the empty values. this is a fine approximation.
    locs_fixed = np.zeros([2,len(cell_ids)])
    for idx in range(len(cell_ids)):
        locs_fixed[0, idx] = np.nanmean(locations[:,idx*2])
        locs_fixed[1, idx] = np.nanmean(locations[:,idx*2+1]) 
    output_array = np.nan*np.ones((9, len(cell_ids)))
    output_array= np.concatenate((circadian_metrics,locs_fixed), axis=0)
    output_array[2,:] *= 360/2/np.pi #transform phase into 360-degree circular format
    output_df = pd.DataFrame(data=output_array,
            columns = list(cell_ids), index=['Rhythmic','CircPeak','Phase','Period','Amplitude',
                                            'Decay','Rsq', 'X', 'Y'])
    output_df.T.to_csv(output_cosine_params, index=True)
    del output_df # clear it
    print str(np.round(timer(),1))+"s"
    
    print "Generating and saving plots: ",
    cellidxs=np.random.randint(len(cell_ids),size=plotcount)
    for cellidx in cellidxs:
        # truly awful syntax
        pf.plot_result(cellidx, raw_times, raw_data, trendlines, 
                    detrended_times, detrended_data, eigenvalues, 
                    final_times, final_data, rhythmic_or_not, 
                    lspers, pgram_data, sine_times, sine_data, r2s,
                    input_folder, data_type)
    print str(np.round(timer(),1))+"s"

    print "All data saved. Run terminated successfully for "+data_type+'.\n'

In [ ]:
plt.figure(figsize=(12,8))
cellidx = 31
plt.plot(detrended_times, detrended_data[:,cellidx])
plt.plot(denoised_times, denoised_data[:,cellidx])
plt.plot(final_times, final_data[:,cellidx])
plt.show()